In [2]:
pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 77.0 MB/s eta 0:00:00


In [3]:
# Question 1
# Data We have
Week1_Data_dem = {'Part 1': 3500, 'Part 2': 3000, 'Part 3': 4000, 'Part 4': 4000, 'Part 5': 2800}
Data_prod = {'Part 1': 40, 'Part 2': 25, 'Part 3': 45, 'Part 4': 50, 'Part 5': 60}
Data_setup = {'Part 1': 8, 'Part 2': 10, 'Part 3': 10, 'Part 4': 8, 'Part 5': 20}

# calculating total production time of each part
def calculate_production_time(demand, Data_prod, Data_setup):
    Total_production_time = {}
    for part, demand in Week1_Data_dem.items():
        production_rate = Data_prod[part]
        setup_time = Data_setup[part]

        # Calculating production time without setup
        production_hours_withoutsetup = demand / production_rate

        # Calculating Total time with setup
        total_hourswithsetup = production_hours_withoutsetup + setup_time

        Total_production_time[part] = total_hourswithsetup
    return Total_production_time

#  The production time
Total_production_time = calculate_production_time(Week1_Data_dem, Data_prod, Data_setup)
for part, time in Total_production_time.items():
    print(f"{part}: The Total production time is {time} hours")

Part 1: The Total production time is 95.5 hours
Part 2: The Total production time is 130.0 hours
Part 3: The Total production time is 98.88888888888889 hours
Part 4: The Total production time is 88.0 hours
Part 5: The Total production time is 66.66666666666666 hours


In [5]:
#Question 2
import pandas as pd
import itertools
from gurobipy import *
Data_File_production = pd.read_excel("data.xlsx", header = 0, index_col=0, sheet_name = "production")
Data_File_setup = pd.read_excel("data.xlsx", header = 0, index_col=0, sheet_name = "setup")
Data_File_demand = pd.read_excel("data.xlsx", header = 0, index_col=0, sheet_name = "demand")
Data_parts = list(Data_File_setup.columns)
Data_machines = list(Data_File_setup.index)
print(Data_parts)
# working time in a week
Data_combine_list = []
for t in itertools.product(Data_machines, Data_parts):
    Data_combine_list.append(t[0]+ "_" +t[1])
Data_setup_cost = {}
for s in Data_combine_list:
    firstData_ = s.split("_")[0]
    secondData_ = s.split("_")[1]
    Data_setup_cost[s] = Data_File_setup.loc[firstData_, secondData_]
Data_regular_hours_machine = {}
for name in Data_machines:
    Data_regular_hours_machine[name] = 120
Data_overtime_hours = {}
for name in Data_machines:
    Data_overtime_hours[name] = 48
# Demand of each part
Data_constraint_left_p_eq = {}
for p in Data_parts:
    Data_production_co = {}
    for v in Data_combine_list:
        firstData_ = v.split("_")[0]
        secondData_ = v.split("_")[1]
        if secondData_ ==p:
            Data_production_co[v] = Data_File_production.loc[firstData_, secondData_]
        else:
            Data_production_co[v] = 0
    Data_constraint_left_p_eq[p] = Data_production_co

# setup time for each machine
Data_setup_time_machine = {}

for m in Data_machines:
    Data_setup_dic = {}
    for v in Data_combine_list:
        firstData_ = v.split("_")[0]
        secondData_ = v.split("_")[1]
        if firstData_ == m:
            Data_setup_dic[v] = Data_File_setup.loc[firstData_, secondData_]
        else:
            Data_setup_dic[v] = 0
    Data_setup_time_machine[m] = Data_setup_dic
# production time for each machine
Data_production_time_machine = {}
for m in Data_machines:
    Data_production_dic = {}
    for v in Data_combine_list:
        firstData_ = v.split("_")[0]
        secondData_ = v.split("_")[1]
        if firstData_ == m:
            Data_production_dic[v] = 1
        else:
            Data_production_dic[v] = 0
    Data_production_time_machine[m] = Data_production_dic
# production_time_machine
def optimize_model(week):
    Data_constraint_right_eq = {}
    for name in Data_parts:
        Data_constraint_right_eq[name] = Data_File_demand.loc[week, name]

    Data_m=Model()

    Data_overtime_production_levels = Data_m.addVars(Data_combine_list, name="overtime")
    Data_setup_levels = Data_m.addVars(Data_combine_list, name="setup", vtype = GRB.BINARY)
    Data_regular_production_levels = Data_m.addVars(Data_combine_list, name="regular")

    overtime_leq = Data_m.addConstrs((quicksum(Data_production_time_machine[Data_m][vp]*Data_overtime_production_levels[vp] for vp in Data_combine_list)
                                           <= Data_overtime_hours[Data_m] for Data_m in Data_machines), name="overtime_leq");


    regular_leq = Data_m.addConstrs((quicksum(Data_production_time_machine[Data_m][vp]*Data_regular_production_levels[vp] for vp in Data_combine_list) + quicksum(Data_setup_time_machine[Data_m][vs]*Data_setup_levels[vs] for vs in Data_combine_list)
                                           <= Data_regular_hours_machine[Data_m] for Data_m in Data_machines), name="regular_leq");

    eq_constraints_set = Data_m.addConstrs((quicksum(Data_setup_levels[v]*Data_constraint_left_p_eq[p][v]*Data_regular_production_levels[v] for v in Data_combine_list) + quicksum(Data_setup_levels[v]*Data_constraint_left_p_eq[p][v]*Data_overtime_production_levels[v] for v in Data_combine_list)
                                           == Data_constraint_right_eq[p] for p in Data_parts ), name="eq_constraints");

    obj = quicksum(Data_overtime_production_levels[vp] for vp in Data_combine_list)
    Data_m.update()
    Data_m.setObjective(obj, GRB.MINIMIZE)
    Data_m.update()
    Data_m.optimize()
    cost = 0
    regular_hours = 0
    for variable in Data_m.getVars():
        if variable.x != 0:
            name = variable.varName
            n1 = name.split("[")[0]
            n2 = name.split("[")[-1][:-1]
            if n1 == "setup":
                value = Data_setup_cost[n2]*variable.x
                cost += Data_setup_cost[n2]*variable.x*100
                regular_hours += Data_setup_cost[n2]*variable.x
            elif n1 == "regular":
                value = variable.x
                cost += 100*variable.x
                regular_hours += variable.x
            else:
                value = variable.x
                cost += 150*variable.x
            print('%s %g' % (variable.varName, value))

    print('Obj: %g' % Data_m.ObjVal)
    print('total cost1: %g' % cost)
    print("regular hours: %g" % regular_hours)
    Data_m.reset()


optimize_model(1)
optimize_model(2)
optimize_model(3)
optimize_model(4)
optimize_model(5)
optimize_model(6)
optimize_model(7)
optimize_model(8)
optimize_model(9)
optimize_model(10)
optimize_model(12)

['Part1', 'Part2', 'Part3', 'Part4', 'Part5']
Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model fingerprint: 0xc34d78de
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 47 columns
Presolve time: 0.00s
Presolved: 68 rows, 46 columns, 173 nonzeros
Variable types: 37 continuous, 9 integer (9 binary)
Found heuristic solution: objective 152.0767288

Root relaxation: objective 1.120894e+02

In [6]:
#Question 3
Data_File_parts = list(Data_File_setup.columns)
Data_File_machines = list(Data_File_setup.index)

Data_File_combine_list = []
for r in itertools.product(Data_File_machines, Data_File_parts):
    Data_File_combine_list.append(r[0] + "_" + r[1])

# Creating dictionary to setup costs
Data_File_setup_cost = {}
for v in Data_File_combine_list:
    firstData_File_, secondData_File_ = v.split("_")
    Data_File_setup_cost[v] = Data_File_setup.loc[firstData_File_, secondData_File_]
regular_hours_machine = {name: 120 for name in Data_File_machines}
overtime_hours = {name: 48 for name in Data_File_machines}
Data_File_constraint_left_p_eq = {}
for p in Data_File_parts:
    production_co = {v: Data_File_production.loc[first, second] if second == p else 0 for v in Data_File_combine_list for first, second in [v.split("_")]}
    Data_File_constraint_left_p_eq[p] = production_co

setup_time_machine = {m: {v: Data_File_setup.loc[first, second] if first == m else 0 for v in Data_File_combine_list for first, second in [v.split("_")]} for m in Data_File_machines}
production_time_machine = {m: {v: 1 if first == m else 0 for v in Data_File_combine_list for first, second in [v.split("_")]} for m in Data_File_machines}

# optimization function for the model of given week
def optimize_model_modified(week):
    Data_File_constraint_right_eq = {name: Data_File_demand.loc[week, name] for name in Data_File_parts}
    Data_File_m = Model()
    Data_File_overtime_production_levels = Data_File_m.addVars(Data_File_combine_list, name="overtime")
    Data_File_setup_levels = Data_File_m.addVars(Data_File_combine_list, name="setup", vtype=GRB.BINARY)
    Data_File_regular_production_levels = Data_File_m.addVars(Data_File_combine_list, name="regular")
    Data_File_max_overtime = Data_File_m.addVar(name="max_overtime")
    Data_File_m.addConstrs((quicksum(production_time_machine[Data_File_m][vp] * Data_File_overtime_production_levels[vp] for vp in Data_File_combine_list) <= Data_File_max_overtime for Data_File_m in Data_File_machines), name="overtime_leq")
    Data_File_m.addConstrs((quicksum(production_time_machine[m][vp] * Data_File_regular_production_levels[vp] for vp in Data_File_combine_list) + quicksum(setup_time_machine[m][vs] * Data_File_setup_levels[vs] for vs in Data_File_combine_list) <= regular_hours_machine[m] for m in Data_File_machines), name="regular_leq")
    Data_File_m.addConstrs((quicksum(Data_File_setup_levels[v] * Data_File_constraint_left_p_eq[p][v] * Data_File_regular_production_levels[v] for v in Data_File_combine_list) + quicksum(Data_File_setup_levels[v] * Data_File_constraint_left_p_eq[p][v] * Data_File_overtime_production_levels[v] for v in Data_File_combine_list) == Data_File_constraint_right_eq[p] for p in Data_File_parts), name="eq_constraints")
    # Objective function
    Data_File_m.setObjective(Data_File_max_overtime, GRB.MINIMIZE)

    # Model optimization
    Data_File_m.update()
    Data_File_m.optimize()

    # results
    for variable in Data_File_m.getVars():
        if 'setup' in variable.varName and variable.x != 0:
            Data_File_machine_part = variable.varName.split('[')[-1].strip(']')
            Data_File_actual_setup_time = Data_File_setup_cost[Data_File_machine_part]
            print(f'{variable.varName}: {Data_File_actual_setup_time}')
        elif variable.x != 0:
            print('%s %g' % (variable.varName, variable.x))

    print('Max Overtime: %g' % Data_File_max_overtime.x)
    Data_File_m.reset()

# Run the model for each week
for week in range(1, 13):
    print(f"Optimizing for Week {week}")
    optimize_model_modified(week)

Optimizing for Week 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 76 columns and 65 nonzeros
Model fingerprint: 0x5b2105b7
Model has 5 quadratic constraints
Variable types: 51 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 65 rows, 71 columns, 175 nonzeros
Presolved model has 20 SOS constraint(s)
Variable types: 51 continuous, 20 integer (20 binary)
Found heuristic solution: objective 32.0886460

Root relaxation: objective 2.287960e+01, 36 iterations, 0.00 seconds (0.00 work units)

   

In [7]:
#Question 4
# Constants for cost
Data_File_cost_per_machine_hour = 30
Data_File_cost_support_personnel_per_hour = 40
Data_File_production = pd.read_excel("data.xlsx", header=0, index_col=0, sheet_name="production")
Data_File_setup = pd.read_excel("data.xlsx", header=0, index_col=0, sheet_name="setup")
Data_File_demand = pd.read_excel("data.xlsx", header=0, index_col=0, sheet_name="demand")
Data_File_parts = list(Data_File_setup.columns)
Data_File_machines = list(Data_File_setup.index)

Data_File_combine_list = []
for r in itertools.product(Data_File_machines, Data_File_parts):
    Data_File_combine_list.append(r[0] + "_" + r[1])
Data_File_setup_cost = {}
for v in Data_File_combine_list:
    firstData_File_, secondData_File_ = v.split("_")
    Data_File_setup_cost[v] = Data_File_setup.loc[firstData_File_, secondData_File_]
Data_File_regular_hours_machine = {name: 120 for name in Data_File_machines}
Data_File_overtime_hours = {name: 48 for name in Data_File_machines}
Data_File_constraint_left_p_eq = {}
for p in Data_File_parts:
    production_co = {v: Data_File_production.loc[first, second] if second == p else 0 for v in Data_File_combine_list for first, second in [v.split("_")]}
    Data_File_constraint_left_p_eq[p] = production_co
Data_File_setup_time_machine = {m: {v: Data_File_setup.loc[first, second] if first == m else 0 for v in Data_File_combine_list for first, second in [v.split("_")]} for m in Data_File_machines}
Data_File_production_time_machine = {m: {v: 1 if first == m else 0 for v in Data_File_combine_list for first, second in [v.split("_")]} for m in Data_File_machines}

# Function to optimize the model for a given week
def optimize_model_cost(week):
    constraint_right_eq = {name: Data_File_demand.loc[week, name] for name in Data_File_parts}
    # Model setup
    m = Model()
    Data_File_overtime_production_levels = m.addVars(Data_File_combine_list, name="overtime")
    Data_File_setup_levels = m.addVars(Data_File_combine_list, name="setup", vtype=GRB.BINARY)
    Data_File_regular_production_levels = m.addVars(Data_File_combine_list, name="regular")
    Data_File_total_overtime = m.addVar(name="total_overtime")
    m.addConstrs((quicksum(production_time_machine[m][vp] * Data_File_overtime_production_levels[vp] for vp in Data_File_combine_list) <= Data_File_total_overtime for m in Data_File_machines), name="overtime_leq")

    m.addConstrs((quicksum(production_time_machine[m][vp] * Data_File_regular_production_levels[vp] for vp in Data_File_combine_list) + quicksum(setup_time_machine[m][vs] * Data_File_setup_levels[vs] for vs in Data_File_combine_list) <= regular_hours_machine[m] for m in Data_File_machines), name="regular_leq")

    m.addConstrs((quicksum(Data_File_setup_levels[v] * Data_File_constraint_left_p_eq[p][v] * Data_File_regular_production_levels[v] for v in Data_File_combine_list) + quicksum(Data_File_setup_levels[v] * Data_File_constraint_left_p_eq[p][v] * Data_File_overtime_production_levels[v] for v in Data_File_combine_list) == constraint_right_eq[p] for p in Data_File_parts), name="eq_constraints")
    Data_File_total_cost = quicksum(Data_File_overtime_production_levels[vp] * Data_File_cost_per_machine_hour for vp in Data_File_combine_list) + Data_File_total_overtime * Data_File_cost_support_personnel_per_hour
    m.setObjective(Data_File_total_cost, GRB.MINIMIZE)
    m.update()
    m.optimize()
    # results
    for variable in m.getVars():
        if variable.x != 0:
            if 'setup' in variable.varName and variable.x > 0.5:
                Data_File_machine_part = variable.varName.split('[')[-1].strip(']')
                Data_File_actual_setup_time = Data_File_setup_cost[Data_File_machine_part]
                print(f'{variable.varName}: {Data_File_actual_setup_time} hours ')
            else:
                print('%s %g' % (variable.varName, variable.x))

    print('Total Cost: %g' % m.objVal)
    m.reset()
for week in range(1, 13):
    print(f"Optimizing for Week {week}")
    optimize_model_cost(week)

Optimizing for Week 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 76 columns and 65 nonzeros
Model fingerprint: 0x20aa8b01
Model has 5 quadratic constraints
Variable types: 51 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [3e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 65 rows, 71 columns, 175 nonzeros
Presolved model has 20 SOS constraint(s)
Variable types: 51 continuous, 20 integer (20 binary)
Found heuristic solution: objective 6096.8427366

Root relaxation: objective 4.347125e+03, 24 iterations, 0.00 seconds (0.00 work units)

 

In [8]:
#Question 5
# Get the machines and parts from the setup sheet
Data_File_parts = list(Data_File_setup.columns)
Data_File_machines = list(Data_File_setup.index)
Data_File_initial_setups = {'Machine1': 'Part1', 'Machine2': 'Part2', 'Machine3': 'Part5', 'Machine4': 'Part3', 'Machine5': 'Part4'}
Data_File_combine_list = []
for r in itertools.product(Data_File_machines, Data_File_parts):
    Data_File_combine_list.append(r[0] + "_" + r[1])

# dictionary for setup costs
Data_File_setup_cost = {}
for v in Data_File_combine_list:
    firstData_File_, secondData_File_ = v.split("_")
    Data_File_setup_cost[v] = Data_File_setup.loc[firstData_File_, secondData_File_]
Data_File_regular_hours_machine = {name: 120 for name in Data_File_machines}
Data_File_overtime_hours = {name: 48 for name in Data_File_machines}
Data_File_constraint_left_p_eq = {}
for p in Data_File_parts:
    production_co = {v: Data_File_production.loc[firstData_File_, secondData_File_] if secondData_File_ == p else 0 for v in Data_File_combine_list for firstData_File_, secondData_File_ in [v.split("_")]}
    Data_File_constraint_left_p_eq[p] = production_co
Data_File_Data_File_setup_time_machine = {m: {v: Data_File_setup.loc[firstData_File_, secondData_File_] if firstData_File_ == m else 0 for v in Data_File_combine_list for firstData_File_, secondData_File_ in [v.split("_")]} for m in Data_File_machines}
production_time_machine = {m: {v: 1 if firstData_File_ == m else 0 for v in Data_File_combine_list for firstData_File_, secondData_File_ in [v.split("_")]} for m in Data_File_machines}

#optimization function of the model for a given week
def optimize_model_with_adjusted_setup(week):
    Data_File_constraint_right_eq = {name: Data_File_demand.loc[week, name] for name in Data_File_parts}
    m = Model()
    Data_File_overtime_production_levels = m.addVars(Data_File_combine_list, name="overtime")
    Data_File_setup_levels = m.addVars(Data_File_combine_list, name="setup", vtype=GRB.BINARY)
    Data_File_regular_production_levels = m.addVars(Data_File_combine_list, name="regular")
    Data_File_total_overtime = m.addVar(name="total_overtime")
    # Constraints
    for machine in Data_File_machines:
        if week == 1:
            m.addConstr((quicksum(production_time_machine[machine][mp] * Data_File_regular_production_levels[mp] for mp in Data_File_combine_list) <=
                         regular_hours_machine[machine]), name=f"regular_leq_{machine}_week1")
        else:
            m.addConstr((quicksum(production_time_machine[machine][mp] * Data_File_regular_production_levels[mp] for mp in Data_File_combine_list) +
                         quicksum(setup_time_machine[machine][mp] * Data_File_setup_levels[mp] for mp in Data_File_combine_list) <=
                         regular_hours_machine[machine]), name=f"regular_leq_{machine}_week{week}")
    m.addConstrs((quicksum(production_time_machine[m][vp] * Data_File_overtime_production_levels[vp] for vp in Data_File_combine_list) <= Data_File_total_overtime for m in Data_File_machines), name="overtime_leq")
    m.addConstrs((quicksum(Data_File_regular_production_levels[v] * Data_File_constraint_left_p_eq[p][v] for v in Data_File_combine_list) + quicksum(Data_File_overtime_production_levels[v] * Data_File_constraint_left_p_eq[p][v] for v in Data_File_combine_list) == Data_File_constraint_right_eq[p] for p in Data_File_parts), name="eq_constraints")
    Data_File_total_cost = quicksum(Data_File_overtime_production_levels[vp] * Data_File_cost_per_machine_hour for vp in Data_File_combine_list) + Data_File_total_overtime * Data_File_cost_support_personnel_per_hour
    m.setObjective(Data_File_total_cost, GRB.MINIMIZE)
    # Model optimization
    m.update()
    m.optimize()
    # Output
    for machine in Data_File_machines:
        Data_File_setup_time = sum(Data_File_setup_cost[machine + "_" + part] for part in Data_File_parts) if week > 1 else 0
        print(f'Machine {machine} Setup Time in Week {week}: {Data_File_setup_time} hours')
    for variable in m.getVars():
        if variable.x != 0:
            print('%s %g' % (variable.varName, variable.x))
    print('Total Cost: %g' % m.objVal)
    m.reset()
for week in range(1, 13):
    print(f"Optimizing for Week {week}")
    optimize_model_with_adjusted_setup(week)


Optimizing for Week 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 15 rows, 76 columns and 75 nonzeros
Model fingerprint: 0x2762711c
Variable types: 51 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [3e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 4e+03]
Presolve removed 1 rows and 56 columns
Presolve time: 0.00s
Presolved: 14 rows, 20 columns, 43 nonzeros
Variable types: 20 continuous, 0 integer (0 binary)

Root relaxation: objective 1.671503e+03, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1671.5029307 

In [9]:
#Question 6
Data_File_parts = list(Data_File_setup.columns)
Data_File_machines = list(Data_File_setup.index)
Data_File_combine_list = list(itertools.product(Data_File_machines, Data_File_parts))
# Creating dictionaries for setup costs, production coefficients, and demand
Data_File_setup_cost = {(m, p): Data_File_setup.loc[m, p] for m, p in Data_File_combine_list}
Data_File_production_coefficient = {(m, p): Data_File_production.loc[m, p] for m, p in Data_File_combine_list}
regular_hours_machine = 120
overtime_hours_machine = 48
# optimization function for the model for two consecutive weeks
def optimize_model_two_weeks(start_week):
    model = Model()
    regular_week1 = model.addVars(Data_File_combine_list, lb=0, name="regular_week1")
    overtime_week1 = model.addVars(Data_File_combine_list, lb=0, name="overtime_week1")
    regular_week2 = model.addVars(Data_File_combine_list, lb=0, name="regular_week2")
    overtime_week2 = model.addVars(Data_File_combine_list, lb=0, name="overtime_week2")
    setup_var = model.addVars(Data_File_combine_list, vtype=GRB.BINARY, name="setup")
    for p in Data_File_parts:
        model.addConstr(sum((regular_week1[m, p] + overtime_week1[m, p] + regular_week2[m, p] + overtime_week2[m, p]) * Data_File_production_coefficient[m, p]
                        for m in Data_File_machines) >= sum(Data_File_demand.loc[w, p] for w in [start_week, start_week + 1]))
    for machine in Data_File_machines:
        for week in [1, 2]:
            regular_var = regular_week1 if week == 1 else regular_week2
            overtime_var = overtime_week1 if week == 1 else overtime_week2
            model.addConstr(sum(regular_var[machine, p] for p in Data_File_parts) <= regular_hours_machine)
            model.addConstr(sum(overtime_var[machine, p] for p in Data_File_parts) <= overtime_hours_machine)
            model.addConstr(sum(regular_var[machine, p] + overtime_var[machine, p] + setup_var[machine, p] * Data_File_setup_cost[machine, p] for p in Data_File_parts) <= regular_hours_machine + overtime_hours_machine)
    total_cost = sum(overtime_week1[m, p] * Data_File_cost_per_machine_hour + overtime_week2[m, p] * Data_File_cost_per_machine_hour for m, p in Data_File_combine_list) + \
                 sum(setup_var[m, p] * Data_File_cost_support_personnel_per_hour for m, p in Data_File_combine_list)
    model.setObjective(total_cost, GRB.MINIMIZE)
    # Model optimization
    model.optimize()
    #results
    for week in [1, 2]:
        print(f"\nWeek {start_week + week - 1} Results:")
        for m, p in Data_File_combine_list:
            regular_var = regular_week1 if week == 1 else regular_week2
            overtime_var = overtime_week1 if week == 1 else overtime_week2
            if regular_var[m, p].x > 0 or overtime_var[m, p].x > 0:
                print(f"Machine {m}, Part {p}: Regular = {regular_var[m, p].x}, Overtime = {overtime_var[m, p].x}")

    print('\nTotal Cost: %g' % model.objVal)
for week in range(1, 12, 2):
    print(f"\nOptimizing for Weeks {week} and {week + 1}")
    optimize_model_two_weeks(week)



Optimizing for Weeks 1 and 2
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 35 rows, 125 columns and 260 nonzeros
Model fingerprint: 0x14201763
Variable types: 100 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [3e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 8e+03]
Presolve removed 27 rows and 112 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 25 nonzeros
Variable types: 13 continuous, 0 integer (0 binary)

Root relaxation: objective 1.585586e+03, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    15

In [18]:
# Question 7
import pandas as pd
import itertools
from gurobipy import *

# Constants for cost
Data_File_cost_per_machine_hour = 30
Data_File_cost_support_personnel_per_hour = 40

# Read the data from the Excel file
Data_File_production = pd.read_excel("data.xlsx", header=0, index_col=0, sheet_name="production")
Data_File_setup = pd.read_excel("data.xlsx", header=0, index_col=0, sheet_name="setup")
Data_File_demand = pd.read_excel("data.xlsx", header=0, index_col=0, sheet_name="demand")

# Get the machines and parts from the setup sheet
Data_File_parts = list(Data_File_setup.columns)
Data_File_machines = list(Data_File_setup.index)

# Generate combinations of machines and parts
Data_File_combine_list = list(itertools.product(Data_File_machines, Data_File_parts))

# Create dictionaries for setup costs, production coefficients, and demand
Data_File_setup_cost = {(m, p): Data_File_setup.loc[m, p] for m, p in Data_File_combine_list}
Data_File_production_coefficient = {(m, p): Data_File_production.loc[m, p] for m, p in Data_File_combine_list}

# Maximum regular and overtime hours per week
regular_hours_machine = 120
overtime_hours_machine = 48

# Function to optimize the model for two consecutive weeks
def optimize_model_two_weeks(start_week):
    # Model setup
    model = Model()

    # Variables for production hours and setup
    regular_week1 = model.addVars(Data_File_combine_list, lb=0, name="regular_week1")
    overtime_week1 = model.addVars(Data_File_combine_list, lb=0, name="overtime_week1")
    regular_week2 = model.addVars(Data_File_combine_list, lb=0, name="regular_week2")
    overtime_week2 = model.addVars(Data_File_combine_list, lb=0, name="overtime_week2")
    setup_var = model.addVars(Data_File_combine_list, vtype=GRB.BINARY, name="setup")

    # Constraints
    # Demand fulfillment over two weeks
    for p in Data_File_parts:
        model.addConstr(sum((regular_week1[m, p] + overtime_week1[m, p] + regular_week2[m, p] + overtime_week2[m, p]) * Data_File_production_coefficient[m, p]
                        for m in Data_File_machines) >= sum(Data_File_demand.loc[w, p] for w in [start_week, start_week + 1]))

    # Machine hours constraints
    for machine in Data_File_machines:
        for week in [1, 2]:
            regular_var = regular_week1 if week == 1 else regular_week2
            overtime_var = overtime_week1 if week == 1 else overtime_week2
            model.addConstr(sum(regular_var[machine, p] for p in Data_File_parts) <= regular_hours_machine)
            model.addConstr(sum(overtime_var[machine, p] for p in Data_File_parts) <= overtime_hours_machine)
            model.addConstr(sum(regular_var[machine, p] + overtime_var[machine, p] + setup_var[machine, p] * Data_File_setup_cost[machine, p] for p in Data_File_parts) <= regular_hours_machine + overtime_hours_machine)
    total_cost = sum(overtime_week1[m, p] * Data_File_cost_per_machine_hour + overtime_week2[m, p] * Data_File_cost_per_machine_hour for m, p in Data_File_combine_list) + \
                 sum(setup_var[m, p] * Data_File_cost_support_personnel_per_hour for m, p in Data_File_combine_list)
    model.setObjective(total_cost, GRB.MINIMIZE)
    model.optimize()

    # Checking for shortfalls and reporting
    shortfalls = []
    for week in [1, 2]:
        current_week = start_week + week - 1
        for p in Data_File_parts:
            total_produced = sum((regular_week1[m, p].x + overtime_week1[m, p].x) * Data_File_production_coefficient[m, p] if week == 1 else (regular_week2[m, p].x + overtime_week2[m, p].x) * Data_File_production_coefficient[m, p] for m in Data_File_machines)
            demand_required = Data_File_demand.loc[current_week, p]
            if total_produced < demand_required:
                shortfall_amount = demand_required - total_produced
                shortfalls.append((current_week, p, shortfall_amount))
                print(f"Week {current_week}, Part {p}: Shortfall of {shortfall_amount}")

    # Output results
    for week in [1, 2]:
        print(f"\nWeek {start_week + week - 1} Results:")
        for m, p in Data_File_combine_list:
            regular_var = regular_week1 if week == 1 else regular_week2
            overtime_var = overtime_week1 if week == 1 else overtime_week2
            if regular_var[m, p].x > 0 or overtime_var[m, p].x > 0:
                print(f"Machine {m}, Part {p}: Regular = {regular_var[m, p].x}, Overtime = {overtime_var[m, p].x}")

    print('\nTotal Cost: %g' % model.objVal)

    # Print summary of shortfalls
    if shortfalls:
        print("\nSummary of Shortfalls:")
        for week, part, shortfall in shortfalls:
            print(f"Week {week}, Part {part}: Shortfall of {shortfall}")
    else:
        print("\nNo shortfalls detected.")

# Run the model for each pair of weeks
for week in range(1, 12, 2):  # Assuming 12 weeks in total
    print(f"\nOptimizing for Weeks {week} and {week + 1}")
    optimize_model_two_weeks(week)


Optimizing for Weeks 1 and 2
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 35 rows, 125 columns and 260 nonzeros
Model fingerprint: 0x14201763
Variable types: 100 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [3e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 8e+03]
Presolve removed 27 rows and 112 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 25 nonzeros
Variable types: 13 continuous, 0 integer (0 binary)

Root relaxation: objective 1.585586e+03, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    15